_Code Cell 1: jupyter helpers_

In [ ]:
# jupyter helpers

# To get multiple outputs from one cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
from IPython.display import HTML, Markdown, Image
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))


def add_div(div_class='info', div_start='Tip:', 
            div_text='Some tip here', output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output, but the cell mode is still 'code',
    not 'markdown'.
    Workaround: After running the function, click on the new cell, press ESC, 
                type 'm', then run the new cell.
    If `output_string=False`, the output is displayed in an new cell with the 
    code cell visible.
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    """
    accepted = ['info', 'warning', 'danger']
    div_class = div_class.lower()
    if div_class not in accepted:
        msg = f'<div class="alert"><b>Wrong class:&nbsp;</b> `div_start` not in: {accepted}.</div>'
        return Markdown(msg)
    
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}&nbsp;&nbsp;</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div)

    
def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    div = f'<div style="{style}">{title}</div>'
    #return HTML('<div style="{}">{}</div>'.format(style, title))
    return get_ipython().set_next_input(div, 'markdown')


_Code Cell 2: project helpers_

In [ ]:
# project helpers

import sys
from pathlib import Path
from pprint import pprint as pp

def sys_info():
    frmt = "\nListing from sys_info():\n"
    frmt += "Python ver: {}\nPython env: {}\n"
    frmt += "OS:         {}\nCurrent dir: {}\n"
    print(frmt.format(sys.version, 
                      Path(sys.prefix).name,
                      sys.platform,
                      Path.cwd()))


def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    newp = Path(this_path).as_posix() # no str method (?)
    if up:
        newp = Path(this_path).parent.as_posix()

    msg = 'Path already in sys.path'
    if newp not in sys.path:
        sys.path.insert(1, newp)
        msg = 'Path added to sys.path'
    print(msg)

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


# Filtered dir() for method discovery:
def filter_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work 
              together to perform search on non-dunder methods (default).
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]

# alias:
fdir = filter_dir

def get_project_dirs(which: list,
                     nb_folder='notebooks',
                     use_parent=True):
    '''Create folder(s) named in `which` at the parent level.'''
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder) or use_parent:
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

#DIR_DATA, 
DIR_IMG, DIR_TMP = get_project_dirs(['images', '_temp'])


# autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')

%autoreload 2

#..................
sys_info()

no_wmark = False
try:
    %load_ext watermark
    #%watermark
except ModuleNotFoundError:
    no_wmark = True

if not no_wmark:
    print("\nListing from watermark ext:")
    %watermark
    #-iv

---
---
# Testing modules
---

# No dry-run for conda env create!
### see [Issue7495](https://github.com/conda/conda/issues/7495)

In [ ]:
from new_conda_env import envir, processing as proc
from new_conda_env.main import generate_parser

In [ ]:
conda_env_parser = generate_parser()
conda_env_parser.prog
print(conda_env_parser.description)

args = conda_env_parser.parse_args("-old_ver 3.10.8 -new_ver 3.9 -dotless_ver 1 -env_to_clone ds310 -new_env_name ds39".split())
args

In [ ]:
conda_vir = envir.CondaEnvir(old_ver=args.old_ver,
                             new_ver=args.new_ver, 
                             dotless_ver=args.dotless_ver,
                             env_to_clone=args.env_to_clone,
                             new_env_name=args.new_env_name,
                             kernel=args.kernel,
                             display_new_yml=args.display_new_yml,
                             debug=args.debug)

fdir(conda_vir)

In [ ]:
conda_vir.create_new_env_yaml()

# Call example at the cmd or terminal window:
`python -m new_conda_env.main -old_ver 3.10 -new_ver 3.9 -dotless_ver 1 -env_to_clone ds310 -new_env_name ds39`